In [10]:
import pandas as pd
import geopandas as geopd

import numpy as np
import random
from sklearn.model_selection import train_test_split

import gym
from gym.wrappers import TimeLimit
# import time

import warnings
warnings.filterwarnings('ignore')
import itertools
import matplotlib.pyplot as plt

# from gym.spaces import Dict,Box,Discrete

In [11]:
import ray
from ray import tune
from ray.rllib.agents import dqn
from ray.tune.registry import register_env
from typing import Dict
# import ray
# from ray import tune
from ray.rllib.env import BaseEnv
from ray.rllib.policy import Policy
# from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks

In [12]:
## Env params:

## import data
nb_of_data = 'Manhattan' 

data  = pd.read_csv(f'./Data/demand_{nb_of_data}.csv')
distance_data = pd.read_csv(f'./Data/distance_info_{nb_of_data}.csv')
max_episode_steps = 200
max_client = 100

env_config = {'demand_data' : data,
              'distance_data':distance_data,
              'client_limit':max_client,
              'customers_per_taxi':2,
              'max_timestep': max_episode_steps,
             'number_taxis':1}
del data, distance_data
dict_results = {}

env_name = f"{env_config['number_taxis']}_TaxiAgent_{env_config['customers_per_taxi']}_passangers_{nb_of_data}"
print(env_config['demand_data'].Demand.sum())

126646


## Env

In [34]:
class Multi_passanger_taxi(gym.Env):
    def __init__(self,env_config):
        ## Get data from input
        self.data = env_config['demand_data'].sort_values(by=['Demand'],ascending=False,ignore_index=True)
        self.distance_data = env_config['distance_data']
        # self.max_timestep =  env_config['max_timestep']
        self.n = env_config['customers_per_taxi'] # max number of customer per taxi

        self.locations =  np.unique(self.distance_data.PULocationID.values)
        self.nb_locations = len(self.locations)

        self.locations = dict(zip(range(self.nb_locations),self.locations))
        self.location_to_index = dict((v, k) for k, v in self.locations.items())

        self.customers=0
        self.custumer_count = self.data.Demand.sum() 
        self.customers_limit = min(self.custumer_count,env_config['client_limit'])
        self.current_client_count=0
        
        self.agents =[f'taxi_{i}' for i in range(env_config['number_taxis'])]
        
        self.action_space = self.get_action_space()

        # Define Observation space 
        self.observation_space = self.get_obs_space()
        # ## Initialise the space  
        self.reset()    
        
    def check_empty_positions(self,state_obs):
        empty_position = np.where(state_obs==-1)
        return min(empty_position,default=None)

        
    def get_distane(self,PU_location,DO_location):
            distance = self.distance_data[(self.distance_data.PULocationID==PU_location) & 
                                          (self.distance_data.DOLocationID==DO_location)]['distance'].values
            return distance[0]
        

    def get_action_space(self):
        action_dict = {'move':gym.spaces.Discrete(self.nb_locations),
                                                'pickup' : gym.spaces.Discrete(1)}
        
        return gym.spaces.Dict({agent:gym.spaces.Dict(action_dict) for agent in self.agents})
    
    
    def get_obs_space(self):
        single_position_obs = {'position': gym.spaces.Box(low = np.array([0]),high=np.array([self.nb_locations-1]),dtype=int),
                      'state':gym.spaces.Box(low = -1,high=self.nb_locations-1,shape=(self.n,),dtype=int),}
        return gym.spaces.Dict({agent:gym.spaces.Dict(single_position_obs) for agent in self.agents})
    
    
    
    def pick_up_reward(self,observation):
        state_obs = observation['state']
        empty_location = self.check_empty_positions(state_obs)
        # print('state', state_obs, 'empty location', empty_location)
        if empty_location is None or len(empty_location)==0:
            return 0, observation
        else:
            taxi_location = self.locations[observation['position'][0]]
            if taxi_location in self.demand_dict.keys(): ## Current location has a client:                   
                destination_location = self.demand_dict[taxi_location].pop(0)
                observation['state'][empty_location[0]] = self.location_to_index[destination_location]
                if self.demand_dict[taxi_location]==[]:
                    self.demand_dict.pop(taxi_location)
                return 10,observation
        return 0,observation
            
            
            
    def move_reward(self,destination_id,observation):
        taxi_location = self.locations[observation['position'][0]]
        destination = self.locations[destination_id]
        if destination_id in observation['state']:
            reward = 0
            destination_indexes, = np.where(observation['state']==destination_id)
            for destination_index in destination_indexes:
                observation['state'][destination_index]=-1
                self.customers+=1
                reward += 100
            return reward, observation
        else:
            return -10,observation
            # if self.check_empty_positions(observation['state']) is None:
            #     return -self.get_distane(taxi_location,destination)*10,observation
            # else: 
            #     return -self.get_distane(taxi_location,destination)*30,observation

            
    def single_step(self, action,observation):
        pu_reward ,move_reward =0,0
        # observation = self.state['taxi_0']
        # action = action['taxi_0']
        # if action['pickup']: ## If action is pick up client:
        pu_reward, observation = self.pick_up_reward(observation)   
        if observation['position'][0]==action['move']: #and action['pickup']==0:
            move_reward =-10
        else:
            move_reward ,observation = self.move_reward(action['move'],observation)
        # print(action, 'pick_up rew',pu_reward, 'move rew',move_reward, 'obs', observation)
        # Return step information
        return observation, pu_reward+move_reward
        


    def step(self, action):
        done = False
        reward =0
        for agent in self.agents:
            taxi_observation = self.state[agent]
            taxi_action = action[agent]
            taxi_observation, agent_reward = self.single_step(taxi_action,taxi_observation)
            self.state[agent] = taxi_observation
            reward += agent_reward
            
        self.time_step +=1

        if self.customers == self.customers_limit or len(self.demand_dict)==0:
            done=True

        # Set placeholder for info
        info = {'customers_dropped': self.customers,
               'done': done,
               'reward': reward}

        # Return step information
        return self.state, reward/len(self.agents), done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.time_step = 0 
        self.customers=0
        # self.taxi_path =[]
        self.state = {agent:{'position':np.random.randint([0],[self.nb_locations]),
                            'state': -np.ones(self.n)} for agent in self.agents}
        self.demand_dict = {pu :[] for pu in self.data.PU_LocationID}
        for index in range(len(self.data)):
            row_index = self.data.loc[index]
            self.demand_dict[row_index.PU_LocationID]+=[row_index.DO_LocationID for _ in range(row_index.Demand)]

        return self.state



## Wrapper

In [35]:
class Obs_Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Dict), \
            "Should only be used to wrap Discrete envs."
        self.old_observation_space = self.observation_space
        self.agent = self.old_observation_space.spaces.keys()
        
        self.observation_space, self.shape = self.create_space()
        
    def create_space(self):
        # nb_agent = len(self.agent)
        dict_value1 = list(self.old_observation_space.spaces.values())[0].spaces.values()
        high,low,shape =[],[],0
        for boxes in dict_value1:
            b_high,b_low, b_shape = boxes.high, boxes.low, boxes.shape
            high= np.append(high,b_high)
            low = np.append(low,b_low)
            shape+=b_shape[0]
        high,low = np.array([high for _ in self.agent]), np.array([low for _ in self.agent])
        shape = (len(self.agent),shape)
        return gym.spaces.Box(high=high,low=low,dtype=int), shape
        
    def observation(self, obs):
        new_obs = np.zeros(self.shape)
        for id_agent, agent in enumerate(self.agent):
            new_obs[id_agent][0] = obs[agent]['position'] 
            new_obs[id_agent][1:] = obs[agent]['state'] 
        return new_obs

In [36]:
class Action_Wrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.Dict), \
            "Should only be used to wrap Discrete envs."
        self.old_action_space = self.action_space
        self.agent = list(self.old_action_space.spaces.keys())
        self.action_space = self.create_space()
        
        
    def create_space(self):
        nb_agent = len(self.agent)
        dict_value1 = list(self.old_action_space.spaces.values())[0].spaces.values()
        shape = np.array([boxes.n for boxes in dict_value1])
        # print('shape',shape)
        action_agent = [[i,j] for i in range(shape[0]) for j in range(shape[1])]
        # print('action',action_agent)
        str_exe = '[('+''.join([f'i{i},' for i in range(nb_agent)])+') '+' '.join([f'for i{i} in {action_agent}' for i in range(nb_agent)])+']'
        # print(str_exe)
        self.all_action_combo = eval(str_exe)
        # print(self.all_action_combo)
        return gym.spaces.Discrete(len(self.all_action_combo))
        
    def action(self, action):
        # print(action)
        # print(self.agent)
        # print(self.all_action_combo)
        action = self.all_action_combo[action]
        # print(action)
        new_action = {}
        for index, elem in enumerate(action):
            new_action[self.agent[index]]={'move' : elem[0],'pickup': elem[1]}
        return new_action

## Callbacks

In [37]:
class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
        episode.custom_metrics["nb_customers_dropped"] = []
        # episode.user_data["nb_customers_picked"] = []

    def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
        info_dict = episode.last_info_for()
        episode.custom_metrics["nb_customers_dropped"].append(info_dict['customers_dropped'])
        
    def on_train_result(self, trainer, result: dict, **kwargs):
        result["callback_ok"] = True



## env register

In [38]:
# dqn.APEX_DEFAULT_CONFIG


# 'gamma': 0.99,
# 'lr': 0.0005,
# 'clip_rewards': None,
# 'lr_schedule': None,
# 'adam_epsilon': 1e-08,

# 'learning_starts': 50000,

# dqn.DEFAULT_CONFIG
# 'gamma': 0.99,
# 'learning_starts': 1000
#  'lr': 0.0005,

In [39]:
def env_multi_creator(env_config):
    env=TimeLimit(Action_Wrapper(Obs_Wrapper(Multi_passanger_taxi(env_config))),env_config['max_timestep'])# return an env instance
    # env=TimeLimit(Multi_passanger_taxi(env_config),env_config['max_timestep'])# return an env instance

    env.seed(1042)
    return env


In [40]:
register_env(env_name, env_multi_creator)

## test 

In [41]:
config = {
    "env": env_name,
    'env_config': env_config,
    'framework': 'torch',
    'callbacks':MyCallbacks,
    # 'gpu':None,
    "lr": 0.0005,
     # "num_gpus_per_worker": 0,
    'gamma': 0.9,
    'seed':1042
}
# trainer_config['env_config']=env_config
# trainer_config['framework']='torch'
client_dropped_criteria = 10*env_config['number_taxis']*env_config['customers_per_taxi']

stop_criteria = {'agent_timesteps_total':100000}




In [ ]:
# tune.run() allows setting a custom log directory (other than ``~/ray-results``)
# and automatically saving the trained agent
import time
ray.shutdown()
ray.init()
register_env(env_name, env_multi_creator)
t1 = time.time()
analysis = ray.tune.run(
    'DQN',
    config=config,
    # local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_at_end=True)

time.time()-t1

Trial name,status,loc
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,PENDING,


(DQNTrainer pid=61530) 2022-04-07 11:39:04,663	INFO simple_q.py:153 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQNTrainer pid=61530) 2022-04-07 11:39:04,663	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQNTrainer pid=61530) 2022-04-07 11:39:08,021	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(DQNTrainer pid=61530) 2022-04-07 11:39:08,023	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Trial name,status,loc
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530


Trial name,status,loc
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530


Trial name,status,loc
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530,40,1143.2,40000,2449.4,18960,-2000,181.18


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530,40,1143.2,40000,2449.4,18960,-2000,181.18


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530,40,1143.2,40000,2449.4,18960,-2000,181.18


Result for DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000:
  agent_timesteps_total: 41000
  callback_ok: true
  custom_metrics:
    nb_customers_dropped_max: 182
    nb_customers_dropped_mean: 36.75
    nb_customers_dropped_min: 0
  date: 2022-04-07_11-58-28
  done: false
  episode_len_mean: 180.47
  episode_media: {}
  episode_reward_max: 18960.0
  episode_reward_mean: 2490.8
  episode_reward_min: -2000.0
  episodes_this_iter: 6
  episodes_total: 220
  experiment_id: 830c55294ce54476ae25520abe6a38c6
  hostname: Wyames-MacBook-Pro.local
  info:
    last_target_update_ts: 40816
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_lr: 0.0005
          grad_gnorm: 0.7390322089195251
          max_q: 239.79965209960938
          mean_q: 66.03931427001953
          min_q: 14.457752227783203
        mean_td_error: -37.394248962402344
        model: {}
        td_error:
        - -39.163429260253906
        - -

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_1_TaxiAgent_2_passangers_Manhattan_fc40e_00000,RUNNING,127.0.0.1:61530,41,1158.04,41000,2490.8,18960,-2000,180.47


In [ ]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
# checkpoints = analysis.get_trial_checkpoints_paths(
#     trial=analysis.get_best_trial("episode_reward_mean", mode='max'),
#     metric="episode_reward_mean")
checkpoints = analysis.get_last_checkpoint()

agent = dqn.DQNTrainer(
    config={'env_config': env_config, 'framework': 'torch'}, env=env_name)
agent.restore(checkpoints)

## test algo

In [43]:
env = env_multi_creator(env_config)
to_print=True
num_episodes=10

env.reset()
total_reward = []
# box_flag = isinstance(env.observation_space,gym.spaces.Box)
for episode in range(num_episodes):
    env.reset()
    state = env.reset()
    state_0 = state.copy()
    done = False
    ep_reward = 0
    step = 0
    if to_print:
        print("****************************************************")
        print("EPISODE ", episode)

    while not done:
        action = agent.compute_action(state)

        # action = env.action_space.sample()

#
        new_state, reward, done, info = env.step(action)
        # print(new_state)
        ep_reward +=reward
        step+=1

        if done:
            total_reward.append(ep_reward)
            if to_print:
                # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
                # num_transported= list(state_0[1:]).count(1) - list(new_state[1:]).count(1)
                print('Initial state : ', state_0, ' Final state : ',new_state)

                # We print the number of step it took.
                print(f"Number of steps {step}, customers transported are {env.customers}, the total reward is {ep_reward}")

            break
        state = new_state
        # state = tuple(new_state) if box_flag else new_state

if to_print:
    print('Mean score: %.3f of %i games!'%(np.mean(total_reward), num_episodes))
# return np.mean(total_reward)

****************************************************
EPISODE  0
Initial state :  [[12. -1. -1.]]  Final state :  [[12. 11. 11.]]
Number of steps 200, customers transported are 10, the total reward is -830.0
****************************************************
EPISODE  1
Initial state :  [[53. -1. -1.]]  Final state :  [[53. 13. 13.]]
Number of steps 200, customers transported are 6, the total reward is -1290.0
****************************************************
EPISODE  2
Initial state :  [[31. -1. -1.]]  Final state :  [[31. 59. 59.]]
Number of steps 200, customers transported are 4, the total reward is -1520.0
****************************************************
EPISODE  3
Initial state :  [[17. -1. -1.]]  Final state :  [[17. 51. 51.]]
Number of steps 200, customers transported are 10, the total reward is -830.0
****************************************************
EPISODE  4
Initial state :  [[56. -1. -1.]]  Final state :  [[56. 57. 57.]]
Number of steps 200, customers transported 

## train RLlib

### train ppo

In [ ]:
# ray.shutdown()
# ray.init()

# register_env("Multi_passanger_env", env_multi_creator)
# env_choice = "Multi_passanger_env"
# training_iter = 10
# # DEfine training parameters
# trainer_config = ppo.DEFAULT_CONFIG.copy()
# trainer_config['env_config']=env_config ##input to the envi
# # trainer_config['framework']='torch'
# trainer_config["clip_rewards"]=300
# trainer_config["seed"]=1042
# trainer_config['lr'] = 0.0001
# trainer_config['callbacks'] = MyCallbacks
# trainer_config['vf_clip_param']=50
# trainer = ppo.PPOTrainer(trainer_config, env=env_choice)
# ep_rew =[]
# nb_cust = []
# for i in range(training_iter):
#    # Perform one iteration of training the policy with PPO
#     result = trainer.train()
#     ep_rew.append(result['episode_reward_mean'])
#     nb_cust.append(result['custom_metrics']['nb_customers_dropped_mean'])
#     if (i+1) % 10 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)
        
# dict_results[str(trainer)]={'episode_reward_mean':ep_rew,
#                             'number_customers':nb_cust}

### train dqn

In [ ]:
# ray.shutdown()
# ray.init()

# register_env("Multi_passanger_env", env_multi_creator)
# env_choice = "Multi_passanger_env"
# # training_iter = 50
# # # DEfine training parameters
# trainer_config = {}
# trainer_config['env_config']=env_config ##input to the envi
# # trainer_config['framework']='torch'
# trainer_config["clip_rewards"]=300
# # trainer_config['lr'] = 0.0001
# trainer_config['callbacks'] = MyCallbacks
# trainer = dqn.DQNTrainer(trainer_config, env=env_choice)
# ep_rew =[]
# nb_cust = []
# for i in range(training_iter):
#    # Perform one iteration of training the policy with PPO
#     result = trainer.train()
#     ep_rew.append(result['episode_reward_mean'])
#     nb_cust.append(result['custom_metrics']['nb_customers_dropped_mean'])
#     if i % 10 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)
        
# dict_results[str(trainer)]={'episode_reward_mean':ep_rew,
#                             'number_customers':nb_cust}

### train pg

In [ ]:
# ray.shutdown()
# ray.init()

# register_env("Multi_passanger_env", env_multi_creator)
# # DEfine training parameters
# trainer_config = pg.DEFAULT_CONFIG.copy()
# trainer_config['env_config']=env_config ##input to the envi
# # trainer_config['framework']='torch'
# trainer_config["clip_rewards"]=300
# # trainer_config['lr'] = 0.0001
# trainer_config['callbacks'] = MyCallbacks
# # Train the model
# trainer = pg.PGTrainer(trainer_config, env=env_choice);
# ep_rew =[]
# nb_cust = []
# for i in range(training_iter):
#    # Perform one iteration of training the policy with PPO
#     result = trainer.train()
#     if i>0:
#         ep_rew.append(result['episode_reward_mean'])
#         nb_cust.append(result['custom_metrics']['nb_customers_dropped_mean'])
#     if i % 10 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)
        
# dict_results[str(trainer)]={'episode_reward_mean':ep_rew,
#                             'number_customers':nb_cust}

### train A3C

In [ ]:
# ray.shutdown()
# ray.init()

# register_env("Multi_passanger_env", env_multi_creator)
# # DEfine training parameters
# trainer_config = a3c.DEFAULT_CONFIG.copy()
# trainer_config['env_config']=env_config ##input to the envi
# # trainer_config['framework']='torch'
# trainer_config['callbacks'] = MyCallbacks
# # trainer_config['lr'] = 0.0001
# # Train the model
# trainer = a3c.A3CTrainer(trainer_config, env=env_choice);
# ep_rew =[]
# nb_cust = []
# for i in range(training_iter):
#    # Perform one iteration of training the policy with PPO
#     result = trainer.train()
#     if i>0:
#         ep_rew.append(result['episode_reward_mean'])
#         nb_cust.append(result['custom_metrics']['nb_customers_dropped_mean'])
#     if i % 10 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)
        
# dict_results[str(trainer)]={'episode_reward_mean':ep_rew,
#                             'number_customers':nb_cust}